In [ ]:
# Attention score로부터 Importance 구하기 *
# Attention score는 주어진 문장 길이의 제곱으로 구해진 Matrix 형태인데, 어떻게 단어별로 중요도를 구할 수 있을까? 라는 의문이 생깁니다. 각 단어의 중요도를 계산하기 위해 입력 시퀀스의 다른 모든 단어에서 각 단어에 대한 Attention Score를 합산할 수 있습니다. (보통, 이는 한 단어의 embeding을 거친 feature vector를 의미할 수 있습니다) 한 단어에 대한 Attention score의 합산 결과는 중요성을 나타내는 단일 점수로 여겨질 수 있습니다.
# 다음은 코드의 예입니다.

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential


vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

word_index = tf.keras.datasets.imdb.get_word_index() # get {word : index}
index_word = {v : k for k,v in word_index.items()} # get {index : word}
index_word[0] = '<PAD>'
index_word[1] = '<START>'
index_word[2] = '<UNK>'

(x_train, y_train), (x_valid, y_valid) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_valid = tf.keras.preprocessing.sequence.pad_sequences(x_valid, maxlen=maxlen)

# token된 데이터 영문으로 출력해보기
DataIndex = 5
" ".join([index_word[idx] for idx in x_train[DataIndex]])
y_train[DataIndex]

EmbeddingDim = 128
WordLen = 200

inputs = tf.keras.layers.Input(shape=(WordLen,))
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=EmbeddingDim)(inputs)  # input_dim을 vocab_size로 변경
attn_output, weight = tf.keras.layers.MultiHeadAttention(num_heads=5, key_dim=4, use_bias=True)(
    embedding_layer, embedding_layer, return_attention_scores=True)
# 나머지 코드...

Model_withWeight = tf.keras.Model(inputs=inputs, outputs=[weight])

attention_weights = Model_withWeight.predict(x_valid[:2])
print(attention_weights.shape)   # (2, 5, 200, 200)

import numpy as np
importance_scores = np.sum(attention_weights, axis=3)
print(importance_scores)